In [1]:
%matplotlib notebook
from sympy import *
import matplotlib.pyplot as plt

I first replicate my hand calculations to make sure that they are correct:

In [2]:
Y2,Z2,Y3,Z3 = symbols('Y2,Z2,Y3,Z3')
eqdpi = [(Z2,1/Y2),
         (Z3,1/Y3)]
G1,G2,G3,G4,C1,C2,K,s = symbols('G1,G2,G3,G4,C1,C2,K,s')
eqdpa = [(Y2,G1+G2+G3+s*C1),
         (Y3,G2+G4+s*C2)]
L1,L2,Delta = symbols('L1,L2,Delta')
eqloops = [(L1,G2**2*Z2*Z3),
           (L2,-K*G2*G3*Z2*Z3)]

Now the denomiantor:

In [3]:
d_T = ( Y2*Y3*(1-L1-L2)/C1/C2 ) .subs(eqloops) .subs(eqdpi) . subs(eqdpa) .simplify()
d_T

(-G2**2 + G2*G3*K + (C2*s + G2 + G4)*(C1*s + G1 + G2 + G3))/(C1*C2)

In [4]:
d_T_p = poly(d_T,s)
a2,a1,a0 = symbols('a2,a1,a0')
[a2,a1,a0] = d_T_p.all_coeffs()
[a2,a1,a0]

[1,
 (C1*G2 + C1*G4 + C2*G1 + C2*G2 + C2*G3)/(C1*C2),
 (G1*G2 + G1*G4 + G2*G3*K + G2*G3 + G2*G4 + G3*G4)/(C1*C2)]

In [5]:
P1 = -K*G1*Z2*G2*Z3
D1 = 1
n_T = ( Y2*Y3*P1*D1/C1/C2 ) .subs(eqloops) .subs(eqdpi) . subs(eqdpa) .simplify()
n_T

-G1*G2*K/(C1*C2)

So far it is the same as my manual solution. Let's calculate the squared versions of wp and qp and check the simplified solution. (A hint to understand the eqqp line: an equation in SymPy is just written as one term that is supposed to be zero. Therefore, x=y is written as [-x+y].)

In [6]:
wp,qp,wp2,qp2,k = symbols('wp,qp,wp2,qp2,k')
wp2 = a0
eqqp = [ -wp2/qp2 + a1**2 ]
qp2 = solve( eqqp, [qp2] )[qp2]
qp2

C1*C2*(G1*G2 + G1*G4 + G2*G3*K + G2*G3 + G2*G4 + G3*G4)/(C1**2*G2**2 + 2*C1**2*G2*G4 + C1**2*G4**2 + 2*C1*C2*G1*G2 + 2*C1*C2*G1*G4 + 2*C1*C2*G2**2 + 2*C1*C2*G2*G3 + 2*C1*C2*G2*G4 + 2*C1*C2*G3*G4 + C2**2*G1**2 + 2*C2**2*G1*G2 + 2*C2**2*G1*G3 + C2**2*G2**2 + 2*C2**2*G2*G3 + C2**2*G3**2)

In [7]:
eqT0 = [ -k*wp2 + n_T ]
T0 = solve( eqT0, [k] )[k]
T0

-G1*G2*K/(G1*G2 + G1*G4 + G2*G3*K + G2*G3 + G2*G4 + G3*G4)

In [8]:
G,C = symbols('G,C')
eqequal = [(G1,G),(G2,G),(G3,G),(G4,G),(C1,C),(C2,C)]
wp2e = wp2 .subs(eqequal) .simplify()
wp2e

G**2*(K + 5)/C**2

In [9]:
qp2e = qp2 .subs(eqequal) .simplify()
qp2e

K/25 + 1/5

In [10]:
T0e = T0 .subs(eqequal) .simplify()
T0e

-K/(K + 5)

Still the same. Can we design it with equal components? Let's try with qp2:

In [11]:
Ks = solve( -25 + qp2e, [K])[0]
Ks

620

Now we call the frequency G/C = 2 pi fx:

In [12]:
fx = symbols('fx')
subsG = [(G,solve( -2*pi*fx + G/C , G) [0])]
subsG

[(G, 2*pi*C*fx)]

In [13]:
wp2s = wp2e .subs([(K,Ks)]) .subs(subsG)
wp2s

2500*pi**2*fx**2

This shall be 2kHz, so:

In [14]:
fxs = solve( -(2*pi*2.0e3)**2 + wp2s , fx )[1]
fxs

80.0000000000000

So the frequency G/C is rather low. Let's take 1k resistors in order to not have too much noise:

In [15]:
Cval = N(1/(1.0e3*2*pi*fxs))
Cval

1.98943678864869e-6

So C is 2uF, all resistor values are 1k, and the feedback resistor KR4 is 620k. Unfortunately, the DC gain is

In [16]:
T0enum = T0e .subs([(K,Ks)])
T0enum

-124/125

I see no way to give this a DC gain of 0dB. That would not even be possible with an ideal amplifier -K without input resistance, i.e. G4=0:

In [17]:
T0 .subs([(G4,0)]) .subs(eqequal) .simplify()

-K/(K + 2)

In [18]:
T0 .subs([(G4,0)]) .subs(eqequal) .simplify() .subs([(K,Ks)])

-310/311

But I guess T0e is close enough to 0dB :)

In [19]:
N(20*log(-T0enum,10))

-0.0697665569164269